In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hydiexe/dataset-fix")

print("Path to dataset files:", path)

100%|██████████| 1.94G/1.94G [00:08<00:00, 246MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/hydiexe/dataset-fix/versions/1


In [ ]:
import os
import shutil
# 2. Definisi Path
source_path = '/root/.cache/kagglehub/datasets/hydiexe/dataset-fix/versions/1/dataset_batik_fix/raw_dataset'
# Perhatikan spasi pada "BATIK _GENERATOR" sesuai request Anda
destination_path = '/content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset'

# 3. Proses Copy
print(f"Source: {source_path}")
print(f"Destination: {destination_path}")

# Cek apakah source ada
if not os.path.exists(source_path):
    print("❌ Error: Path asal tidak ditemukan. Pastikan dataset KaggleHub sudah ter-download.")
else:
    # Cek apakah folder tujuan sudah ada
    if os.path.exists(destination_path):
        print(f"⚠️ Folder tujuan '{destination_path}' sudah ada.")
        print("Hapus folder lama atau ganti nama tujuan jika ingin menimpa.")
    else:
        print("⏳ Sedang menyalin dataset... (Mungkin memakan waktu tergantung ukuran)")
        try:
            # copytree menyalin seluruh isi folder dan membuat folder baru dengan nama 'dataset'
            shutil.copytree(source_path, destination_path)
            print("✅ Sukses! Dataset berhasil disalin.")
            print(f"Jumlah file: {len(os.listdir(destination_path))}")
        except Exception as e:
            print(f"❌ Terjadi kesalahan saat menyalin: {e}")

Source: /root/.cache/kagglehub/datasets/hydiexe/dataset-fix/versions/1/dataset_batik_fix/raw_dataset
Destination: /content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset
⏳ Sedang menyalin dataset... (Mungkin memakan waktu tergantung ukuran)
✅ Sukses! Dataset berhasil disalin.
Jumlah file: 20


In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# 2. Path Dataset
dataset_path = '/content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset/batik_dayak'

# 3. Proses Pengecekan
print(f"🔍 Memeriksa folder: {dataset_path}\n" + "-"*40)

total_images = 0
all_image_paths = []
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.webp')

# Cek apakah folder utama ada
if not os.path.exists(dataset_path):
    print(f"❌ Error: Folder '{dataset_path}' tidak ditemukan!")
else:
    # Loop melalui semua folder dan subfolder
    for root, dirs, files in os.walk(dataset_path):
        # Filter hanya file gambar
        image_files = [f for f in files if f.lower().endswith(valid_extensions)]
        count = len(image_files)

        if count > 0:
            folder_name = os.path.basename(root)
            # Jika root adalah folder dataset utama, beri nama "Root"
            if folder_name == os.path.basename(dataset_path):
                folder_name = "Folder Utama"

            print(f"📁 {folder_name:<30} : {count} gambar")
            total_images += count

            # Simpan path gambar untuk sampling nanti
            for img in image_files:
                all_image_paths.append(os.path.join(root, img))

    print("-" * 40)
    print(f"✅ TOTAL GAMBAR DITEMUKAN: {total_images}")


🔍 Memeriksa folder: /content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset/batik_dayak
----------------------------------------
📁 Folder Utama                   : 150 gambar
----------------------------------------
✅ TOTAL GAMBAR DITEMUKAN: 150


In [ ]:
import os
from PIL import Image
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt

# --- PATH DATASET ---
DATASET_FOLDER = "/content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset"
# --------------------

def recursive_eda(root_folder):
    print(f"📂 Menganalisis seluruh dataset di: {root_folder} ...")

    valid_extensions = {'.jpg', '.jpeg', '.png', '.webp', '.bmp'}
    all_image_paths = []

    # 1. KUMPULKAN SEMUA PATH GAMBAR (REKURSIF)
    # os.walk akan menelusuri setiap subfolder
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if os.path.splitext(file)[1].lower() in valid_extensions:
                full_path = os.path.join(root, file)
                all_image_paths.append(full_path)

    total_files = len(all_image_paths)
    if total_files == 0:
        print("❌ Folder kosong atau tidak ada gambar!")
        return

    print(f"✅ Ditemukan total {total_files} gambar. Memulai pengecekan detail...")

    modes = []
    sizes = []
    corrupt_files = []
    aspect_ratios = []

    # 2. PROSES ANALISIS
    for filepath in tqdm(all_image_paths):
        try:
            with Image.open(filepath) as img:
                # Cek Mode Warna
                modes.append(img.mode)
                # Cek Ukuran
                w, h = img.size
                sizes.append((w, h))
                aspect_ratios.append(w / h)
        except Exception:
            corrupt_files.append(filepath)

    # 3. LAPORAN
    print("\n" + "="*40)
    print("           HASIL ANALISIS DATASET")
    print("="*40)
    print(f"Total Gambar Terbaca : {len(modes)}")
    print(f"File Rusak/Corrupt   : {len(corrupt_files)}")

    if corrupt_files:
        print("-> Cek list 'corrupt_files' untuk menghapusnya.")

    print("\n--- 1. Distribusi Channel Warna ---")
    mode_counts = Counter(modes)
    for mode, count in mode_counts.items():
        if mode == "RGB":
            status = "✅ AMAN"
        elif mode == "RGBA":
            status = "⚠️ BAHAYA (Alpha Channel - Perlu Convert ke RGB)"
        elif mode == "L":
            status = "⚠️ BAHAYA (Grayscale - Perlu Convert ke RGB)"
        elif mode == "P":
            status = "⚠️ BAHAYA (Palette - Perlu Convert ke RGB)"
        elif mode == "CMYK":
            status = "⚠️ BAHAYA (Print Mode - Perlu Convert ke RGB)"
        else:
            status = "⚠️ Cek Manual"

        print(f"  - {mode:<5} : {count:<5} file -> {status}")

    print("\n--- 2. Statistik Resolusi ---")
    widths = [s[0] for s in sizes]
    heights = [s[1] for s in sizes]

    if widths and heights:
        print(f"  - Terkecil : {min(widths)}x{min(heights)} px")
        print(f"  - Terbesar : {max(widths)}x{max(heights)} px")
        print(f"  - Rata-rata: {int(sum(widths)/len(widths))}x{int(sum(heights)/len(heights))} px")

    # Cek Konsistensi Persegi (Penting untuk GAN)
    square_images = sum(1 for w, h in sizes if w == h)
    not_square = len(sizes) - square_images
    print(f"\n--- 3. Konsistensi Bentuk (Shape) ---")
    print(f"  - Kotak (Square)    : {square_images} file")
    print(f"  - Persegi Panjang   : {not_square} file (Akan di-crop saat preprocessing)")

    # Warning resolusi kecil
    too_small = sum(1 for w, h in sizes if w < 256 or h < 256)
    if too_small > 0:
        print(f"\n[!] PERINGATAN: Ada {too_small} gambar resolusi rendah (< 256px).")
    else:
        print("\n[OK] Semua gambar memiliki resolusi yang cukup.")

# Jalankan Fungsi
recursive_eda(DATASET_FOLDER)

📂 Menganalisis seluruh dataset di: /content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset ...
✅ Ditemukan total 3000 gambar. Memulai pengecekan detail...


100%|██████████| 3000/3000 [00:08<00:00, 354.47it/s]


           HASIL ANALISIS DATASET
Total Gambar Terbaca : 3000
File Rusak/Corrupt   : 0

--- 1. Distribusi Channel Warna ---
  - RGB   : 2957  file -> ✅ AMAN
  - P     : 7     file -> ⚠️ BAHAYA (Palette - Perlu Convert ke RGB)
  - RGBA  : 36    file -> ⚠️ BAHAYA (Alpha Channel - Perlu Convert ke RGB)

--- 2. Statistik Resolusi ---
  - Terkecil : 84x100 px
  - Terbesar : 3648x3904 px
  - Rata-rata: 622x709 px

--- 3. Konsistensi Bentuk (Shape) ---
  - Kotak (Square)    : 471 file
  - Persegi Panjang   : 2529 file (Akan di-crop saat preprocessing)

[!] PERINGATAN: Ada 424 gambar resolusi rendah (< 256px).


In [ ]:
import os
import numpy as np
from PIL import Image
from collections import Counter

# --- KONFIGURASI PATH ---
DATASET_FOLDER = "/content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset"
# ------------------------

def folder_by_folder_eda(root_path):
    print(f"🚀 Memulai Analisis Per-Folder di: {root_path}\n")

    valid_extensions = {'.jpg', '.jpeg', '.png', '.webp', '.bmp'}

    # Variabel Global untuk Rangkuman Akhir
    global_modes = []
    global_sizes = []
    global_corrupt = []
    total_folders_checked = 0

    # Loop menggunakan os.walk untuk menelusuri setiap folder
    for current_root, dirs, files in os.walk(root_path):

        # 1. Filter file gambar di folder yang sedang aktif
        image_files = [f for f in files if os.path.splitext(f)[1].lower() in valid_extensions]

        # Jika folder ini tidak punya gambar, skip saja
        if not image_files:
            continue

        total_folders_checked += 1
        folder_name = os.path.basename(current_root)

        # Variabel Lokal (Per Folder)
        local_modes = []
        local_widths = []
        local_heights = []
        local_corrupt = 0

        print("-" * 60)
        print(f"📂 FOLDER: {folder_name}")
        print(f"   📍 Path: {current_root}")

        # 2. Analisis setiap gambar dalam folder ini
        for filename in image_files:
            filepath = os.path.join(current_root, filename)
            try:
                with Image.open(filepath) as img:
                    # Simpan data lokal
                    local_modes.append(img.mode)
                    local_widths.append(img.size[0])
                    local_heights.append(img.size[1])

                    # Simpan data global
                    global_modes.append(img.mode)
                    global_sizes.append(img.size)
            except Exception:
                local_corrupt += 1
                global_corrupt.append(filepath)

        # 3. Print Statistik Folder Ini
        count = len(local_modes)
        print(f"   📊 Jumlah Gambar Valid : {count}")

        if local_corrupt > 0:
            print(f"   ❌ File Rusak          : {local_corrupt}")

        if count > 0:
            # Mode Warna Folder Ini
            modes_str = ", ".join([f"{m}: {c}" for m, c in Counter(local_modes).items()])
            print(f"   🎨 Format Warna        : {modes_str}")

            # Rata-rata Resolusi Folder Ini
            avg_w = int(sum(local_widths) / count)
            avg_h = int(sum(local_heights) / count)
            min_dim = min(min(local_widths), min(local_heights))

            print(f"   📏 Resolusi Rata-rata  : {avg_w}x{avg_h} px")
            print(f"   🔻 Resolusi Terkecil   : {min(local_widths)}x{min(local_heights)} px")

            # Warning khusus per folder
            if "RGBA" in local_modes:
                print("   ⚠️  WARNING: Folder ini berisi gambar Transparan (RGBA).")
            if min_dim < 256:
                print("   ⚠️  WARNING: Ada gambar resolusi rendah (<256px) di folder ini.")

    # --- RANGKUMAN GLOBAL ---
    print("\n" + "="*60)
    print("REKAPITULASI DATASET (GLOBAL)")
    print("="*60)
    print(f"Total Folder Berisi Gambar : {total_folders_checked}")
    print(f"Total Semua Gambar         : {len(global_modes)}")
    print(f"Total File Rusak           : {len(global_corrupt)}")

    if global_modes:
        print("\nDistribusi Mode Warna Global:")
        for mode, count in Counter(global_modes).items():
            print(f"  - {mode} : {count}")

    print("="*60)

# Jalankan Fungsi
folder_by_folder_eda(DATASET_FOLDER)

🚀 Memulai Analisis Per-Folder di: /content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset

------------------------------------------------------------
📂 FOLDER: batik_wahyu_tumurun
   📍 Path: /content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset/batik_wahyu_tumurun
   📊 Jumlah Gambar Valid : 150
   🎨 Format Warna        : RGB: 150
   📏 Resolusi Rata-rata  : 547x815 px
   🔻 Resolusi Terkecil   : 335x321 px
------------------------------------------------------------
📂 FOLDER: batik_dayak
   📍 Path: /content/drive/MyDrive/FP AI LANJUT/BATIK _GENERATOR/dataset/batik_dayak
   📊 Jumlah Gambar Valid : 150
   🎨 Format Warna        : RGB: 140, P: 4, RGBA: 6
   📏 Resolusi Rata-rata  : 379x377 px
   🔻 Resolusi Terkecil   : 88x101 px
   ⚠️  WARNING: Folder ini berisi gambar Transparan (RGBA).
   ⚠️  WARNING: Ada gambar resolusi rendah (<256px) di folder ini.
------------------------------------------------------------
📂 FOLDER: batik_sidomulyo
   📍 Path: /content/drive/MyDrive/FP AI 